In [39]:
#import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import wave
import xml.etree.ElementTree as ET

In [124]:
rootdir= "/home/megha/Desktop/c"

# 1) # make new directory and convert the wav file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "folk_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
            
#2) extract information from .fln file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        #df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            #print(file)
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text                        
                            df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df1 = df1.append(df_a,ignore_index = True) 
                #print(df1.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df2 = df2.append(df_b,ignore_index = True)
                #print(df2.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df3 = df3.append(df_c,ignore_index = True)
                #print(df3.size) 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        df = df.drop(df[df.col3 ==""].index)        
        endtime = []        
        string_final = []
        count = 0
        count1 = 10
#--------------------------------------------------------------------------------------
        for i in range(len(df)):
            x = df.get_values()[i]
            if(i%5 == 0):
                time = []
                string = []
                if(i%5 == 0):
                    string.append(x[2])
                    time.append(float(x[1]))
                else:
                    string[-1] = string[-1] + x[2]
                    time[-1] = float(x[1])
                endtime.append(time[-1])
                string_final.append(string[-1])          
#----------------------------------------------------------------------------------------
        starttime = []
        starttime.append(0.0)
        starttime = starttime + endtime[:-1]
        string_final = [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"folk.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
        

                
                                


0
(447, 4)
chunking the audio folk_1_0.flac
chunking the audio folk_1_1.flac
chunking the audio folk_1_2.flac
chunking the audio folk_1_3.flac
chunking the audio folk_1_4.flac
chunking the audio folk_1_5.flac
chunking the audio folk_1_6.flac
chunking the audio folk_1_7.flac
chunking the audio folk_1_8.flac
chunking the audio folk_1_9.flac
chunking the audio folk_1_10.flac
chunking the audio folk_1_11.flac
chunking the audio folk_1_12.flac
chunking the audio folk_1_13.flac
chunking the audio folk_1_14.flac
chunking the audio folk_1_15.flac
chunking the audio folk_1_16.flac
chunking the audio folk_1_17.flac
chunking the audio folk_1_18.flac
chunking the audio folk_1_19.flac
chunking the audio folk_1_20.flac
chunking the audio folk_1_21.flac
chunking the audio folk_1_22.flac
chunking the audio folk_1_23.flac
chunking the audio folk_1_24.flac
chunking the audio folk_1_25.flac
chunking the audio folk_1_26.flac
chunking the audio folk_1_27.flac
chunking the audio folk_1_28.flac
chunking the 

In [8]:
rootdir= "F:/Thesis/1/data_wwww/" 

# 1) # make new directory and convert the wav file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "folk_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
            
#2) extract information from .fln file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        #df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            #print(file)
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text                        
                            df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df1 = df1.append(df_a,ignore_index = True) 
                #print(df1.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df2 = df2.append(df_b,ignore_index = True)
                #print(df2.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:
                            if(child.tag == "time"):
                                time_array.append(child.attrib['time'])
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))
                        df3 = df3.append(df_c,ignore_index = True)
                #print(df3.size) 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        #df = df.drop(df[df.C ==""].index)  
        check = df.iloc[::8, :].index
        time1 = []
        time2=  []
        starttime = []
        endtime = []
        string = []
        string_final = []
        for i in range(len(df)):
            x = df.get_values()[i]
            if(i in check):
                time1.append(float(x[0]))
                time2.append(float(x[1]))
                string.append(x[2])
            else:
                time2[-1] = float(x[1])
                string[-1] = string[-1] + x[2]
                if(i%8 == 7):
                    starttime.append(time1[-1])
                    endtime.append(time2[-1])
                    string_final.append(string[-1]) 
        string_final =  [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"folk.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
        

AttributeError: 'float' object has no attribute 'append'

In [40]:
df

,A,B,C
0,0.392,2.136,jetzt muss ich nur noch als erstes
1,2.136,2.736,
2,2.736,3.234,den
3,3.234,4.2,
4,4.2,4.696,
5,4.416,4.696,
6,4.696,6.308,ton testen sagt mal bei test
7,5.576,5.82,
8,5.924,6.857,test
9,6.857,7.352,test


In [54]:
check = df.iloc[::8, :].index
time1 = []
time2=  []
starttime = []
endtime = []
string = []
string_final = []
for i in range(len(df)):
    x = df.get_values()[i]
    if(i in check):
        time1.append(float(x[0]))
        time2.append(float(x[1]))
        string.append(x[2])
    else:
        time2[-1] = float(x[1])
        string[-1] = string[-1] + x[2]
        if(i%8 == 7):
            starttime.append(time1[-1])
            endtime.append(time2[-1])
            string_final.append(string[-1]) 
string_final =  [x.upper() for x in string_final] 
df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})

In [52]:
#df.to_csv("F:/Thesis/1/data_wwww/abc.csv", index=False)

In [55]:
pd.set_option('display.max_colwidth', -1)
df_final

,col1,col2,col3
0,0.392,5.820,JETZT MUSS ICH NUR NOCH ALS ERSTES DEN TON TESTEN SAGT MAL BEI TEST
1,5.924,9.978,TEST TEST TEST TE T
2,9.978,13.200,TEST TE O ES T HA HA GEHT GU
3,12.967,17.720,ÄH DANN K S IM PRINZIP LOSGEHEN OKAY STEHT BEI DIR START TINA
4,17.720,26.620,JA N KAY UND VON DEM START UNTER M START SOZUSAGEN NE UNTER D WORT HM HM HM HM Z DU EINFACH N GRADEN STRICH BIS ZU DIESEM RAD AN
5,26.620,35.780,OKAY UND DANN GEHST DU RUNTER DA ST DASS DU SOZUSAGEN AM ENDE VOM RAD BIST DA UNTER UND ALSO UNTER DEM UNTER DEM RAD VORBEI GENAU UNTER DEM RAD VORBEI
6,35.780,40.262,HM HM OKAY JET BIS DE JA GANZ RU NEE NICHT GANZ RUM OKAY
7,39.860,47.652,ALSO DU GEHST Q RUNTER UND DANN RECHTS SEHR WAIT WEITER AN DEM RAD VORBEI NE MH UND DANN GEHST DU WIEDER HOCH ZUM TO ÄHM
8,47.652,54.186,OKAY WARTE UND DANN AUCH BIS ÜBER DEN TOASTER ÜBER DAS BILD HM HM UND DANN GEHST DU DA DENN NACH
9,54.186,58.981,LINKS HM HM UND DANN ÄH ABER NUR


In [9]:
                for i in range(len(df)):
                    x = df.get_values()[i]
                    if float(x[1]) < count1:
                        if count == 0:
                            string.append(x[2])
                            time.append(float(x[1]))
                        else:
                            string[-1] = string[-1] + x[2]
                            time[-1] = float(x[1])
                        count = count + 1
                    else:                       
                        string.append(x[2])
                        time.append(float(x[1]))
                        string[-1] = string[-1] + x[2]
                        time[-1] = float(x[1])
                        count1 = count1 + 10
                        count = 0
                        endtime.append(time[-1])
                        string_final.append(string[-1])

['abc ghj']

In [88]:
df_final

,col1,col2,col3
0,0.000,6.857,TEST
1,6.857,10.866,TEST TE
2,10.866,15.540,ÄH DANN K S IM PRINZIP LOSGEHEN
3,15.540,18.980,JA N KAY UND VON DEM START UNTER M START SOZU...
4,18.980,32.730,OKAY UND DANN GEHST DU RUNTER DA ST DASS DU S...
5,32.730,38.628,HM HM OKAY JET BIS DE JA GANZ RU
6,38.628,46.362,ALSO DU GEHST Q RUNTER UND DANN RECHTS SEHR W...
7,46.362,54.186,OKAY WARTE UND DANN AUCH BIS ÜBER DEN TOASTER...
8,54.186,60.606,BIS DU SIEHST DOCH DAS MOTORRAD NE
9,60.606,73.231,ALSO DU ZIEHST VOM VOM TOASTER DA ÜBER M TOAS...


In [24]:
ass = df.iloc[::5, :].index

In [25]:
10 in ass

True

In [ ]:
      
        
        i = 0
        while i < len(df):
            #for i in range(len(df)):
            print(i)
            x = df.get_values()[i]
            if float(x[1]) < count1:
                if count == 0:
                    string.append(x[2])
                    time.append(float(x[1]))
                else:
                    string[-1] = string[-1] + x[2]
                    time[-1] = float(x[1])
                if i == len(df)-1:
                    endtime.append(time[-1])
                    string_final.append(string[-1])
                count = count + 1
            else:  
                #string[-1] = string[-1] + x[2]
                #time[-1] = float(x[1])
                #print(i)
                i = i-1
                #print(i)
                endtime.append(time[-1])
                string_final.append(string[-1])
                count1 = count1 + 10
                count = 0 
            i = i+1
            print("-------------------------------------")

In [120]:
pd.set_option('display.max_colwidth', -1)
df

,A,B,C
0,0.0,1.34,okay lukas start mer oder
1,1.34,1.85,
2,1.85,2.505,gut starte ma
3,2.505,3.21,
4,3.21,7.615,dann würd ich einfach sagen wie fandsch n schn wie fandesch deine stunde einfach mal so deine einschätzung fanschst gut oder
5,7.615,8.295,
6,7.995,8.295,
7,8.115,8.552,wa
8,8.552,8.77,
9,8.77,9.29,gu ja


In [114]:
#--------------------------------------cleaning folder---------------------------------------------
if True: # False True
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        #print(Dir)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            print(file)
            if(file.endswith(".xml") or file.endswith(".WAV") or file.endswith(".fln") or file.endswith(".html")):
                os.remove(rootdir+"/"+Dir+"/"+file)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            print(file)
            if(file.endswith(".fln") or file.endswith(".wav")):
                os.remove(rootdir+"/"+Dir+"/temp/"+file)
            else:
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
        os.rmdir(rootdir+"/"+Dir+"/temp/")

FOLK_E_00144.xml
FOLK_E_00144_SE_01_A_01_DF_01.WAV
FOLK_E_00144_SE_01_T_01.fln
FOLK_E_00144_SE_01_T_01.html
FOLK_E_00144_e_doc.html
FOLK_S_00361.xml
FOLK_S_00362.xml
temp
FOLK_E_00144_SE_01_T_01.fln
folk.trans.txt
folk_1.wav
folk_1_0.flac
folk_1_1.flac
folk_1_2.flac
folk_1_3.flac
folk_1_4.flac
folk_1_5.flac
folk_1_6.flac
folk_1_7.flac
folk_1_8.flac
folk_1_9.flac
folk_1_10.flac
folk_1_11.flac
folk_1_12.flac
folk_1_13.flac
folk_1_14.flac
folk_1_15.flac
folk_1_16.flac
folk_1_17.flac
folk_1_18.flac
folk_1_19.flac
folk_1_20.flac
folk_1_21.flac
folk_1_22.flac
folk_1_23.flac
folk_1_24.flac
folk_1_25.flac
folk_1_26.flac
folk_1_27.flac
folk_1_28.flac
folk_1_29.flac
folk_1_30.flac
folk_1_31.flac
folk_1_32.flac
folk_1_33.flac
folk_1_34.flac
folk_1_35.flac
folk_1_36.flac
folk_1_37.flac
folk_1_38.flac
folk_1_39.flac
folk_1_40.flac
folk_1_41.flac
folk_1_42.flac
folk_1_43.flac
folk_1_44.flac
folk_1_45.flac
folk_1_46.flac
folk_1_47.flac
folk_1_48.flac
folk_1_49.flac
folk_1_50.flac
folk_1_51.flac
folk_